In [1]:
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import time

# The Ising Model: An NP-Complete Ferromagnetic Model
### This project will focus on the implementation and optimization of the Ising model, a model intended to represent the interactions of ferromagnetic atoms in a condensed material.

In [2]:
def pickler(filename, data_to_save):
    """
    Writes Python data structure in memory to file on disk
    """
    with open(filename, 'wb') as file:
        pickle.dump(data_to_save, file)

def unpickler(filename):
    """
    Reads data on disk to Python data structure in memory
    """
    with open(filename, 'rb') as file:
        output = pickle.load(file)
    return output

def ising_generator(n, random_state=0):
    """
    Generates an nxn array of randomly assigned spin up (1) or spin down (0) elements to represent a 2D ferromagnetic material
        Inputs: 
            n, integer
        Outputs: 
            ising_arr, nxn np.array()
        Errors:
            AssertError: raised if input is not of type int.
    """
    assert type(n) is int, "Input must be of type int."
    ising_arr = np.random.randint(2,size = (n,n))
    ising_arr[ising_arr == 0] = -1
    return ising_arr

@jit(nopython = True)
def config_energy(ising_arr, J):    
    """
    Inputs:
        ising_arr: np.ndarray, randomly distributed 0s and 1s in square matrix
        J: Interaction term, determines if anti-ferromagnetic (+1) or ferromagnetic (-1) or non-interacting (0)
    Outputs:
        final_energy: total energy associated with configuration of ising_arr
    Errors:
        AssertError: ising_arr must be a square array
    """
    assert J == 1 or J == -1 or J == 0 #Validate input for J
    assert ising_arr.shape[0] == ising_arr.shape[1] #Assert square input array
    n = ising_arr.shape[0]
    num_interactions = 0
    hamiltonian = 0 #init hamiltonian summation
    for i in range(n):
        for j in range(n):
            ## Only need to calculate interaction with element below and element to the right to prevent double counting ferromagnetic interactions
            hamiltonian += -J * ising_arr[i][j] * ising_arr[ (i + 1) % n ][j] #Lower interaction, periodic boundary condition is maintained by the remainder calculation in the index
            hamiltonian += -J * ising_arr[i][j] * ising_arr[i][ (j +1 ) % n ] #Right interaction, periodic boundary condition is maintained by the remainder calculation in the index
    hamiltonian = hamiltonian / n**2 #Normalization of total hamiltonian energy to energy/site
    magnetism = np.sum(ising_arr) / n**2 #Total magnetism normalized to magnetization/site
    return hamiltonian, magnetism

@jit(nopython = True)
def single_flip(ising_arr, J, i, j, init_energy):
    """
    Computationally efficient method of calculating change in energy by flipping a site's magnetization in MCMC
    Inputs:
        ising_arr: square np.array() from ising_generator()
        J: Interaction term, int of either -1, 0, 1
        i: int, row index to flip
        j: int, column index to flip
        init_energy: float-like, energy of configuration before flip
    Outputs: 
        delta_e: Potential change in configurational energy as a result of the element's flip in magnetism
        new_e: Configurational energy following the flip in magnetism
    """
    n = ising_arr.shape[0]
    delta_e = 0
    delta_e += -J * ising_arr[i][j] * ising_arr[(i+1) % n][j] #upper interaction
    delta_e += -J * ising_arr[i][j] * ising_arr[(i-1) % n][j] #lower interaction
    delta_e += -J * ising_arr[i][j] * ising_arr[i][(j + 1) % n] #right interaction
    delta_e += -J * ising_arr[i][j] * ising_arr[i][(j - 1) % n] # left interaction

    new_e = init_energy + delta_e

    return delta_e, new_e

@jit(nopython = True)
def markovchain_montecarlo(ising_arr, mcmc_iterations, equilibriating_iterations, savename):
    """
    Wrapper for a single Markov Chain Monte Carlo optimization of a 2D Ising lattice.
    Inputs:
        ising_arr: Array to optimize from ising_generator function
        mcmc_iterations: int, number of MCMC iterations to run following equilibriation
    Outputs:
        energies: array of configurational energies/step
        magnetism: array of total magnetization/step
    """
    assert ising_arr.shape[0] == ising_arr.shape[1] #must be square array
    n = ising_arr.shape[0] 
    #Equilibriation
    for i in range(equilibriating_iterations):
        temp = 0
        #### Call MCMC optimization for equilbriating_iterations steps
        #### Do not record energies/magnetism to memory
    for i in range(mcmc_iterations):
        #### Call MCMC optimization for mcmc_iterations steps
        #### Write energies/magnetism to memory 
    
    #Write run to 

config1 = ising_generator(10)
begin = time.time()
en1 = config_energy(config1, -1)[0]
single_flip(config1, -1, 2, 4, en1)
end = time.time()
print(f"Time to run: {end-begin} seconds") 

IndentationError: expected an indented block (2577841294.py, line 105)

Time to run: 5.91278076171875e-05 seconds
